In [ ]:
import numpy as np
import random
import math
from copy import deepcopy

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


%matplotlib notebook

In [ ]:
# Parameters

reduced_density = 0.9
reduced_temperature = 0.9
num_particles = 50

beta = 1 / reduced_temperature
# box_length = np.cbrt(num_particles/reduced_density)
box_length = 10.0
cutoff = 3.0
cutoff2 = np.power(cutoff, 2)

# 
num_steps = 100
max_displacement = 1

In [ ]:
# Generate initial state

# coordinate = (0.5 - np.random.rand(num_particles, 3)) * box_length

spacing = int(np.cbrt(num_particles) + 1)
x_vector = np.linspace(0.0, box_length, spacing)
y_vector = np.linspace(0.0, box_length, spacing)
z_vector = np.linspace(0.0, box_length, spacing)
grid  = np.meshgrid(x_vector, y_vector, z_vector)
stack = np.vstack(grid)
coordinates = stack.reshape(3, -1).T
excess = len(coordinates) - num_particles
coordinates = coordinates[:-excess]
coordinates *= 0.95

#coordinates = np.loadtxt("lj_sample_config_periodic1.txt", skiprows=2, usecols=(1,2,3))


In [ ]:
# Loop over steps
initial_coordinates = deepcopy(coordinates)

for step in range(num_steps):
    
    # Randomly pick one of N particles
    random_particle = random.randrange(num_particles)
    
    # Generate random x, y, z displacement
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)
    
    # Modify coordinate of Nth particle by generated displacements
    original_coordinates = coordinates[random_particle, :]
    new_coordinates = deepcopy(original_coordinates)
    
    new_coordinates[0] += x_rand
    new_coordinates[1] += y_rand
    new_coordinates[2] += z_rand
    
    
    # Compute energy difference from particle movement. 
    
    for i_particle in range(0, num_particles):
        # Compute original energy of particle with system.
        new_energy = 0
        old_energy = 0
        
        if i_particle != random_particle:
            # Compute distance between particles using minimum image convention
            r_i = coordinates[i_particle, :]
            r_j = original_coordinates
            rij = r_i - r_j
            rij = rij - box_length * np.round(rij/box_length)
            rij2 = np.sqrt(np.dot(rij, rij))

            # Check if distance is less than the cut-off
            if rij2 < cutoff:
                # If less than cut off, calculate pairwise energy
                sig_by_r6 = np.power(1/rij2, 3)
                sig_by_r12 = np.power(sig_by_r6, 2)
                original_pairwise_energy = 4.0 * (sig_by_r12 - sig_by_r6)
            else:
                # Else, pairwise energy is 0
                original_pairwise_energy = 0
            
            old_energy += original_pairwise_energy


            # Compute new energy of particle with system.

            # Compute distance between particles using minimum image convention
            r_i = coordinates[i_particle, :]
            r_j = new_coordinates
            rij = r_i - r_j
            rij = rij - box_length * np.round(rij/box_length)
            rij2 = np.sqrt(np.dot(rij, rij))

            # Check if distance is less than the cut-off
            if rij2 < cutoff:
                # If less than cut off, calculate pairwise energy
                sig_by_r6 = np.power(1/rij2, 3)
                sig_by_r12 = np.power(sig_by_r6, 2)
                new_pairwise_energy = 4.0 * (sig_by_r12 - sig_by_r6)
            else:
                # Else, pairwise energy is 0
                new_pairwise_energy = 0
            
            new_energy += new_pairwise_energy
        
    # Calculate change in potential energy (requires energy of previous state)
    delta_energy = original_pairwise_energy - new_pairwise_energy
    
    # If negative change in energy, accept move, overwrite coordinates
    # if deltaU >0 check acceptance criteria
    if delta_energy < 0:
        coordinates[random_particle, :] = new_coordinates
    else:
        
        probability_acc = math.exp(-delta_energy*beta)
        
        if probability_acc > random.uniform(0, 1):
            # Move is accepted.
            coordinates[random_particle, :] = new_coordinates
        else:
            # Move is not accepted. Do not update coordinates
            pass
        
    

In [ ]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(111, projection='3d')
ax2.scatter(initial_coordinates[:,0], initial_coordinates[:,1], initial_coordinates[:,2])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(coordinates[:,0], coordinates[:,1], coordinates[:,2])